In [1]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_values
import numpy as np

In [2]:
# Function to establish connection to PostgreSQL
def create_connection(dbname, user, password, host, port):
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)
    return conn


In [3]:
# Function to clean the currency columns and replace 'NaN' with None
def clean_currency(value):
    if pd.isnull(value) or value == 'NA':
        return None
    return float(str(value).replace('$', '').replace(',', '').replace("'", ""))

In [4]:
# Function to convert pandas timestamp to date
def convert_timestamp_to_date(timestamp):
    if pd.isnull(timestamp) or timestamp == 'NA':
        return None
    return timestamp.date()

In [5]:
# Function to ingest data from Excel to PostgreSQL
def ingest_data_from_excel_to_postgres(excel_file, sheet_name, conn):
    # Read Excel data into a DataFrame
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    
    # Convert all 'NA' and 'NaN' to None (NULL in SQL)
    df = df.where(pd.notnull(df), None)

    # Clean the currency and date columns
    for col in ['Billed_amt', 'Paid Amount', 'Check Amount', 'Balance', 'Paid Amount - Dec', 'Adjustment Amount', 'Denied']:
        df[col] = df[col].apply(clean_currency)
        
    for col in ['DOB', 'Date of Service', 'Last Billed Date', 'Date', 'Paid date']:
        df[col] = df[col].apply(convert_timestamp_to_date)
        
     # Prepare the data tuples
    tuples = [tuple([
        row['Sl#'], 
        row['AR Category'], 
        row['status'], 
        row['GK order ID# / AMD Chart #'], 
        row['Accession'], 
        row['Patient_id'], 
        row['Claim_id / AMD Visit #'], 
        row['Patient Name'], 
        row['DOB'], 
        row['Payer Family'], 
        row['Payer'], 
        row['Date of Service'], 
        row['AMD CPT Codes'], 
        row['Billed_amt'], 
        row['Last Billed Date'], 
        row['Date'], 
        row['Aging Days from DOS'], 
        row['Aging Bucket from DOS'], 
        row['TFL Bucket'], 
        row['Paid Amount'], 
        row['Balance'], 
        row['Check#'], 
        row['Billed Status'], 
        row['Pay status'], 
        row['Paid date'], 
        row['Claim status'], 
        row['AR Status'], 
        row['Response'], 
        row['Denial Category'], 
        row['Medi-Cal Claims Remark'], 
        row['GK Location'], 
        row['Report'], 
        row['Billed Amount'], 
        row['Allowed Amount'], 
        row['Paid Amount - Dec'], 
        row['Adjustment Amount'], 
        row['Denied'], 
        row['Remark Code'], 
        row['Check Amount'], 
        row['File name / Remarks'], 
        row['Page#'], 
        row['Comments'], 
        row['Names']
    ]) for index, row in df.iterrows()]

    # SQL query for insertion
    query = """
    INSERT INTO Payment_mapping (sl_no, ar_category, status, gk_order_id_or_amd_chart, accession, patient_id, claim_id_or_amd_visit, patient_name, dob, payer_family, payer, date_of_service, amd_cpt_codes, billed_amt, last_billed_date, date_x, aging_days_from_dos, aging_bucket_from_dos, tfl_bucket, paid_amount, balance, check_no, billed_status, pay_status, paid_date, claim_status, ar_status, response, denial_category, medi_cal_claims_remark, gk_location, report, billed_amount, allowed_amount, paid_amount_dec, adjustment_amount, denied, remark_code, check_amount, file_name_or_remarks, page, comments, names)
    VALUES %s;
    ON CONFLICT (sl_no) DO NOTHING;
    """

    # Insert data into the database
    cursor = conn.cursor()
    try:
        execute_values(cursor, query, tuples)
        conn.commit()
    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()
    finally:
        cursor.close()

In [6]:
if __name__ == "__main__":
    excel_file_path = "C:/Users/DELL/Documents/Dashboard/Payment Posting Till date_for automation.xlsx"  # Update with the path to your excel file
    sheet_name = "Payment Mapping"  # provide the exact sheet
    
    conn = create_connection(dbname="postgres", user="postgres", password="mysecretpassword", host="localhost", port="5432")
    ingest_data_from_excel_to_postgres(excel_file_path, sheet_name, conn)

KeyError: 'Paid Amount'